In [ ]:
import random
import numpy as np
import pulp
import pandas as pd
import multiprocessing as mp
import pickle

DEBUG = 0

def generate(k, n):
    P = np.random.randint(-100, 100, size=(k, n-k))
    I = np.eye(k)
    G = np.hstack((I, P))
    Psi = np.random.choice([-1, 1], size=n-k)  # m dependant for m in M: Psi_=Psi[:m]
    Y = list(range(n))
    a = Y.pop(random.randrange(len(Y)))
    b = Y.pop(random.randrange(len(Y)))
    X = np.array([])  # X starts empty and gets filled (initially m-1=1 at the first iteration of LPsolver)
    return G, P, Psi, a, b, X, Y

def LPsolver(n, k, G, Psi, a, b, X, Y):
    # Define the LP maximization problem
    prob = pulp.LpProblem("Maximize", pulp.LpMaximize)

    # Create decision variables: u_i (assume non-negative)
    U = [pulp.LpVariable(f"u_{i}", lowBound=0) for i in range(k)]
    
    # Objective function: maximize sum_i s0 * g[i] * u_i
    prob += pulp.lpSum(Psi[0] * G[i, a] * U[i] for i in range(k)), "Objective"
    
    for idx, j in enumerate(X):
        j = int(j)
        prob += pulp.lpSum((Psi[idx+1] * G[i, j] - Psi[0] * G[i, a]) * U[i] for i in range(k)) <= 0, f"Constraint1_{j}"
        prob += pulp.lpSum(-Psi[idx+1] * G[i, j] * U[i] for i in range(k)) <= -1, f"Constraint2_{j}"
    
    prob += pulp.lpSum(G[i, b] * U[i] for i in range(k)) == 1, f"Constraint3"
    
    for j in Y:
        j = int(j)
        prob += pulp.lpSum(G[i, j] * U[i] for i in range(k)) <= 1, f"Constraint4_{j}"
        prob += pulp.lpSum(-G[i, j] * U[i] for i in range(k)) <= 1, f"Constraint5_{j}"

    # Solve the LP
    result_status = prob.solve()
    if DEBUG:
        print("Status:", pulp.LpStatus[result_status])
    
    if pulp.LpStatus[result_status] == "Optimal":
        solution_U = [pulp.value(u) for u in U]
        if DEBUG:
            print("Optimal Objective Value:", pulp.value(prob.objective))
            print('G:\n', G)
            print('Ψ', Psi)
            print('a,b', a, b)
        return pulp.value(prob.objective)
    else:
        if DEBUG:
            print("LP did not find an optimal solution.")
        return -1

def solve(n, k):
    M=list(range(n-k+1))[2:]
    print(f'solving n,k = {n,k}')
    print(f'over m = {M}')
    attempts = 0
    bounded = 0
    while True:
        if attempts%5000==0: print(f'{attempts} attempts reached\n{bounded} bounded solutions found')
        G, P, Psi, a, b, X, Y = generate(k, n)
        attempts += 1
        results = np.array([])
        for m in M:
            t = Y.pop(random.randrange(len(Y)))
            X = np.append(X, t)
            X = sorted(X)
            r = LPsolver(n, k, G, Psi[:m], a, b, X, Y)
            if r==-1:break
            bounded += 1
            results = np.append(results, r)
        if r==-1: continue
        print('G:\n',G)
        print('Ψ',Psi)
        print('a,b',a,b)
        print('X',X)
        print('Y',Y)
        return P, results

def ds(n, k, samples):
    # Use multiprocessing Pool to run multiple solve instances in parallel
    with mp.Pool() as pool:
        # Create a list with (n,k) repeated samples times
        args = [(n, k)] * samples
        results_list = pool.starmap(solve, args)
    
    # Combine results into a DataFrame
    df = pd.DataFrame()
    for idx, res in enumerate(results_list):
        if res == -1:
            continue  # Skip failed attempts
        P, results = res
        row_data = {f'val{i}': [results[i]] for i in range(len(results))}
        row_data['P'] = [P]
        df = pd.concat([df, pd.DataFrame(row_data)], ignore_index=True)
        with open(f'data_{n}_{k}_{samples}.pkl', 'wb') as f:
            pickle.dump(df, f)
    return df

if __name__ == "__main__":
    D = ds(10, 4, 500)
    print(D)
